<a href="https://colab.research.google.com/github/danielfrgs/2023_1-Lenguaje_Natural/blob/main/Proyecto_Final/PF_PLN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyecto Final PLN | Clickbait

Se proponen dos tareas principalmente para este reto:

Tarea 1: Sobre la clasificación del tipo de spoiler: La entrada es la publicación de clickbait y el documento vinculado. La tarea es clasificar el tipo de spoiler que justifica la publicación de clickbait (ya sea "phrase", "passage", "multi"). Para cada entrada, se debe generar una salida como `{"uuid": "<UUID>", "spoilerType": "<SPOILER-TYPE>"}` donde `<SPOILER-TYPE>` es una `phrase`, `passage`, or `multi`.

Tarea 2: Sobre la generación de spoilers: La entrada es la publicación de clickbait y el documento vinculado (y, opcionalmente, el tipo de spoiler si su enfoque usa este campo). La tarea es generar el spoiler para la publicación de clickbait. Para cada entrada, se debe generar una salida como `{"uuid": "<UUID>", "spoiler": "<SPOILER>"}` donde `<SPOILER>` es el spoiler de la publicación clickbait.

## Sobre los datos

Los datos vienen en formato JSON Lines (.jsonl) donde cada línea contiene una publicación de clickbait y la versión limpiada manualmente del documento vinculado. Para cada línea, el objetivo es clasificar el tipo de spoiler necesario (tarea 1) y/o generar el spoiler (tarea 2).

Para cada entrada en el conjunto de datos de entrenamiento y validación, están disponibles los siguientes campos:

- `uuid`: El `uuid` de la entrada del conjunto de datos.
- `postText`: El texto de la publicación de clickbait que se va a estropear.
- `targetParagraphs`: El contenido principal de la página web enlazada para clasificar el tipo de spoiler (tarea 1) y generar el spoiler (tarea 2). Consiste en los párrafos del contenido principal extraído manualmente.
- `targetTitle`: El título de la página web enlazada para clasificar el tipo de spoiler (tarea 1) y generar el spoiler (tarea 2).
- `targetUrl`: La URL de la página web vinculada.
- `humanSpoiler`: El spoiler generado por humanos (abstractivo) para la publicación de clickbait de la página web vinculada. Este campo solo está disponible en el conjunto de datos de entrenamiento y validación (no durante la prueba).
- `spoiler`: El spoiler extraído por humanos para la publicación de clickbait de la página web vinculada. Este campo solo está disponible en el conjunto de datos de entrenamiento y validación (no durante la prueba).
- `spoilerPositions`: la posición del spoiler extraído por humanos para la publicación de clickbait de la página web vinculada. Este campo solo está disponible en el conjunto de datos de entrenamiento y validación (no durante la prueba).
etiquetas: el tipo de spoiler (puede ser "frase", "pasaje" o "múltiple") que se clasificará en la tarea 1 (clasificación del tipo de spoiler). Este campo solo está disponible en el conjunto de datos de entrenamiento y validación (no durante la prueba). ).
Algunos campos contienen metainformación adicional sobre la entrada, pero no se usan: `postId`, `postPlatform`, `targetDescription`, `targetKeywords`, `targetMedia`.


## Estado del Arte

Los clickbaits son artículos en línea con títulos engañosos diseñados deliberadamente para atraer a más y más lectores a abrir la página web deseada. Los clickbaits se utilizan para tentar a los visitantes a hacer clic en un enlace en particular, ya sea para monetizar la página de destino o para difundir noticias falsas con fines sensacionalistas. La presencia de clickbaits en cualquier portal agregador de noticias puede generar una experiencia desagradable para los lectores. La detección automática de titulares de clickbaitch de titulares de noticias ha sido un problema desafiante para la comunidad de aprendizaje automático. Se han propuesto muchos métodos para prevenir artículos clickbait en el pasado reciente.

In [2]:
!pip3 install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 7.8 MB/s 
     |████████████████████████████████| 163 kB 69.2 MB/s 
     |████████████████████████████████| 7.6 MB 53.4 MB/s 


In [4]:
!pip install torch torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import tensorflow as tf
from transformers import BertTokenizer

# Tarea 1

In [6]:
df = pd.read_json('train.jsonl', lines=True)

In [7]:
df['targetParagraphs'][0]

['It’ll be just like old times this weekend for Tom Brady and Wes Welker.',
 'Welker revealed Friday morning on a Miami radio station that he contacted Brady because he’ll be in town for Sunday’s game between the New England Patriots and Miami Dolphins at Gillette Stadium. It seemed like a perfect opportunity for the two to catch up.',
 'But Brady’s definition of "catching up" involves far more than just a meal. In fact, it involves some literal "catching" as the Patriots quarterback looks to stay sharp during his four-game Deflategate suspension.',
 '"I hit him up to do dinner Saturday night. He’s like, ‘I’m going to be flying in from Ann Arbor later (after the Michigan-Colorado football game), but how about that morning we go throw?’ " Welker said on WQAM, per The Boston Globe. "And I’m just sitting there, I’m like, ‘I was just thinking about dinner, but yeah, sure. I’ll get over there early and we can throw a little bit.’ "',
 'Welker was one of Brady’s favorite targets for six seas

In [8]:
# Diccionario

# phrase : 0 
# passage : 1
# multi : 2

def numericas(x):
    if x[0] == 'phrase':
        return 0
    elif x[0] == 'passage':
        return 1
    else:
        return 2

df['cat'] = df['tags'].agg(numericas)

In [9]:
df['cat'].value_counts()

0    1367
1    1274
2     559
Name: cat, dtype: int64

### Aproximación por Transformers

Los transformers han demostrado tener amplios usos en PLN, con buenos resultados, incluyendo la clasificación multicategorias, que es lo que estamos buscando ahora.

In [10]:
# Inicializamos el tokenizador base del modelo de Bert
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [11]:
df['text'] = df['targetParagraphs'].apply(lambda x: ' '.join(x))

In [12]:
X_input_ids = np.zeros((len(df), 300))
X_attn_mask = np.zeros((len(df), 300))

In [13]:
def training(df, ids, mask, tokenizer):
    for i, text in tqdm(enumerate(df['text'])):
        token_text = tokenizer.encode_plus(
            text,
            max_length = 300,
            truncation = True,
            padding = 'max_length',
            add_special_tokens = True,
            return_tensors = 'tf'
        )
        ids[i, :] = token_text.input_ids
        mask[i, :] = token_text.attention_mask
    return ids, mask

In [14]:
X_input_ids, X_attn_mask = training(df, X_input_ids, X_attn_mask, tokenizer)

0it [00:00, ?it/s]

In [15]:
# Ahoara vamos a crear un array con las categorías. Siguiendo OneHontedEncoding
labels = np.zeros((len(df), 3))

In [16]:
labels[np.arange(len(df)), df['cat'].values] = 1

In [17]:
# Vamos a crear el conjuto de datos categorizado a partir de nuestrsod atos entrenados:
dataset = tf.data.Dataset.from_tensor_slices((X_input_ids, X_attn_mask, labels))

In [18]:
def mapeo(input_ids, attn_mask, labels):
    return{
        'input_ids' :  input_ids,
        'attention_mask' : attn_mask
        }, labels

dataset = dataset.map(mapeo)

In [19]:
dataset.take(1)

<TakeDataset element_spec=({'input_ids': TensorSpec(shape=(300,), dtype=tf.float64, name=None), 'attention_mask': TensorSpec(shape=(300,), dtype=tf.float64, name=None)}, TensorSpec(shape=(3,), dtype=tf.float64, name=None))>

In [20]:
dataset = dataset.shuffle(10000).batch(16, drop_remainder = True)

In [21]:
p = 0.8
train_size = int((len(df)//16)*p)

train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)

In [22]:
# Ahora vamos a crear el mdoelo

from transformers import TFBertModel

bert_model = TFBertModel.from_pretrained('bert-base-cased')

Downloading:   0%|          | 0.00/527M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [23]:
input_ids = tf.keras.layers.Input(shape = (300, ), name = 'input_ids', dtype = 'int32')
attention_mask = tf.keras.layers.Input(shape = (300, ), name = 'attention_mask', dtype = 'int32')

bert_embds = bert_model.bert(input_ids, attention_mask = attention_mask)[1]
intermediate_layer = tf.keras.layers.Dense(512, activation = 'relu',  name = 'intermediate_layer')(bert_embds)
output_layer = tf.keras.layers.Dense(3, activation = 'softmax', name = 'output_layer')(intermediate_layer)

model = tf.keras.Model(inputs = [input_ids, attention_mask], outputs = output_layer)

In [24]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 300)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 300)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 300,                                           

In [25]:
# Parametros:
optim = tf.keras.optimizers.Adam(learning_rate = 1e-5, decay = 1e-6)
loss_func = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')

In [26]:
model.compile(optimizer=optim, loss = loss_func, metrics = [acc])

In [27]:
hist = model.fit(
    train_dataset,
    validation_data = val_dataset,
    epochs = 1
)

160/160 [==============================] - 192s 1s/step - loss: 1.0445 - accuracy: 0.4246 - val_loss: 1.0241 - val_accuracy: 0.4844


In [28]:
model.save('prueba_clickbait')

In [29]:
trans_model = tf.keras.models.load_model('prueba_clickbait')

In [30]:
df_p = pd.read_json('validation.jsonl', lines=True)

In [31]:
# Diccionario

# phrase : 0 
# passage : 1
# multi : 2

df_p['cat'] = df['tags'].agg(numericas)

df_p['text'] = df['targetParagraphs'].apply(lambda x: ' '.join(x))

In [44]:
def prepare_data(input_text, tokenizer=tokenizer):
    token = tokenizer.encode_plus(
        input_text,
        max_length=300, 
        truncation=True, 
        padding='max_length', 
        add_special_tokens=True,
        return_tensors='tf'
    )
    return {
        'input_ids': tf.cast(token.input_ids, tf.float64),
        'attention_mask': tf.cast(token.attention_mask, tf.float64)
    }

def make_prediction(processed_data, model = trans_model):
    probs = model.predict(processed_data)[0]
    return np.argmax(probs)

In [45]:
prep = df_p['text'].agg(prepare_data)

In [ ]:
predic = prep.agg(make_prediction)

In [49]:
df_p['predict'] = predic

In [52]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

accuracy_score(df_p['cat'], df_p['predict'])

0.47875

In [53]:
confusion_matrix(df_p['cat'], df_p['predict'])

array([[147, 201,   0],
       [ 86, 236,   0],
       [ 51,  79,   0]])